<a href="https://colab.research.google.com/github/abualia4/CloudFast-DL4PuDe/blob/main/system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloud-based Deep Learning System
Inpus stream
1. Web camera or any camera that connect to a web client.
2. Virtual camera using a video recording of crowded event entrance; a video is in the sample directory.

In [1]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Libraries and Inputs

In [2]:
import sys
from google.colab import output
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
from IPython.display import HTML
from scipy.ndimage.filters import gaussian_filter
from IPython.display import Image
from IPython.display import Image, display
import cv2
from google.colab import output
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from IPython.display import display
import csv
import time
from PIL import Image
import os
import matplotlib.pyplot as plt
from argparse import ArgumentParser
from collections import OrderedDict
import cv2
import numpy as np
import torch
np.set_printoptions(threshold=sys.maxsize)
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential,load_model
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import warnings
warnings.filterwarnings('ignore')
 
import tensorflow as tf
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join
import seaborn as sns 

from IPython.display import clear_output
%matplotlib inline

#################################################################################

############################### User Inputs #####################################

#################################################################################
#The path of the system directory
home_path='/content/drive/My Drive/Colab Notebooks/CD4EPD/system/'
sys.path.insert(0,home_path)

#The path of RAFT core directory
sys.path.append('/content/drive/My Drive/Colab Notebooks/CD4EPD/system/RAFT/core')
from raft import RAFT
from utils import flow_viz

#The dimensions of the input image for a classifier.
img_rows, img_cols = 224,224

#The path of the CNN classifier
CNN_path=home_path+"models/ourModel.h5"

#ROI coordinates
roi=[107,55,669,270] #Entrance_2 video example
 
# Rows x columns patches
patch=[2,4] #Entrance_2 video example

#In video_stream function, please determine the width and height of the monitoring window 

# Libraries and Functions

In [ ]:
#Loading the CNN classifier
CNN_model= load_model(CNN_path)
CNN_model.load_weights(CNN_path)

#Image classification
def classify(img):
    img= img[...,::-1]
    
    im_pil = Image.fromarray(img)
    im_resized = im_pil.resize((img_rows, img_cols),resample=0)
    
    img_array =img_to_array(  im_resized )
    image_array_expanded = np.expand_dims(img_array, axis = 0)
     
    group=CNN_model.predict(image_array_expanded,verbose = 0 )
    
    if(group>=0.5):
        return 1
    else:
        return 0

#Determining the coordinates of the patches
def patchBorder(rows, cols, left_top_x,left_top_y,right_bottom_x, right_bottom_y ):
        #global x1
        x1=[]
        #global x2
        x2=[]
        #global y1
        y1=[]
        #global y2
        y2=[]
        
        width=right_bottom_x-left_top_x
        height=right_bottom_y-left_top_y
        for i in range(rows):
            for j in range(cols):
                 
                patch_width=int(width/cols)
                patch_height=int(height/rows)
            
                left_x=left_top_x+patch_width*j
                right_x=left_top_x+patch_width*(j+1)
                left_y=left_top_y+patch_height*i
                right_y=left_top_y+patch_height*(i+1)
                
                x1.append(left_x)
                x2.append(right_x)
                y1.append(left_y)
                y2.append(right_y)
         
        if(x1[0]>x2[0]):
            return(x2,y1,x1,y2)
        else:
            return(x1,y1,x2,y2)   


#Updating ROI coordinates to fit tensor size in RAFT
def new_roi(ratio, roi):
    
    w=roi[2]-roi[0]
    h=roi[3]-roi[1]
    new_w=int(w*ratio)
    new_w=new_w-((new_w%24))
    new_h=int(h*ratio)
    new_h=new_h-((new_h%24))
    
    roi[0]=int(roi[0]*ratio)
    roi[1]=int(roi[1]*ratio)
    roi[2]=roi[0]+new_w
    roi[3]=roi[1]+new_h
   
    return roi

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

#Live video stream from camera
#JavaScript to properly create our live video stream using our webcam as input
def video_stream():

  js = Javascript('''
  /////////////////////////////////////////////////
  ///////////////////User Inputs///////////////////
    var w=960 // window width
    var h=540 // window height
  ////////////////////////////////////////////////
    var w_css=w+'px'
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, w, h);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
     
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = w_css;
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span></span>";
      labelElement = document.createElement('span');
      labelElement.innerText = '';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = w; //video.videoWidth;
      captureCanvas.height = h; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
      
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    } 
    ''')

  display(js)
  
def video_frame(label, bbox):
  
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  
 
  return data

#Visualizing the optical flow vectors, and then generating the annotation mask.
def vizualize_flow(img, flo, save, counter):
   
    img = img[0].permute(1, 2, 0).cpu().numpy()
    flo = flo[0].permute(1, 2, 0).cpu().numpy()
    flo = flow_viz.flow_to_image(flo) 
    flo = cv2.cvtColor(flo, cv2.COLOR_RGB2BGR)
    image_blurred = cv2.blur(src=img, ksize=(10, 10))
    labels=[]
    for patch_no in range (0,len(x1)):  
       patch=flo[y1[patch_no]:y2[patch_no],x1[patch_no]:x2[patch_no]]
       if(classify(patch)==1):
           border=[int(ox1[patch_no]),int(oy1[patch_no]),int(ox2[patch_no]),int(oy2[patch_no])]
           labels.append(border )
           cv2.rectangle(image_blurred, (int(x1[patch_no])+4,int(y1[patch_no])+7), (int(x2[patch_no]-4),int(y2[patch_no])-4), (0,0,255),3)
    name=time.time()      
    cv2.imwrite(home_path+"/storage/"+str(name)+".png", image_blurred)
    return labels

#Frame preprocessing
def frame_preprocess(frame, device):
    frame = torch.from_numpy(frame).permute(2, 0, 1).float()
    frame = frame.unsqueeze(0)
    frame = frame.to(device)
    return frame

#Frames extraction and processing, optical flow vectors estimation and visualization
def inference(args):   
    # get the RAFT model
    model = RAFT(args)    
    # load pretrained weights
    pretrained_weights = torch.load(args.model)   
    save = args.save
    
    if torch.cuda.is_available():
        device = "cuda"
        # parallel between available GPUs
        model = torch.nn.DataParallel(model)
        # load the pretrained weights into model
        model.load_state_dict(pretrained_weights)
        model.to(device)
       
    # change model's mode to evaluation
    model.eval() 
    video_stream()
    label_html = ''
    bbox = ''  
    rows=args.patch[0]
    cols=args.patch[1]
    roi_original=args.roi
    roi_n=new_roi(args.ratio, args.roi)
    # capture the video and get the first frame
    js_reply = video_frame(label_html, bbox)
    frame_1 = js_to_image(js_reply["img"])
    counter =1
    #fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width=frame_1.shape[1]
    frame_height=frame_1.shape[0]
    dim = (int(frame_width*args.ratio),int(frame_height*args.ratio))
    frame_1 = cv2.resize(frame_1, dim, interpolation = cv2.INTER_AREA)
    #Patches borders on the original frames
    global ox1
    ox1=[]
    global ox2
    ox2=[]
    global oy1
    oy1=[]
    global oy2
    oy2=[] 
    ox1,oy1,ox2,oy2=patchBorder(rows, cols, args.roi[0],args.roi[1],args.roi[2],args.roi[3] )
    #Cropped ROI
    frame_1=frame_1[roi_n[1]:roi_n[3],roi_n[0]:roi_n[2]]
    global x1
    x1=[]
    global x2
    x2=[]
    global y1
    y1=[]
    global y2
    y2=[] 
    x1,y1,x2,y2=patchBorder(rows, cols, 0,0,int(frame_1.shape[1]), int(frame_1.shape[0]) )
   
    with torch.no_grad():
        begin=time.time()
        js_reply = video_frame(label_html, bbox)
        frame_1 = js_to_image(js_reply["img"])
        frame_1=frame_1[roi_n[1]:roi_n[3],roi_n[0]:roi_n[2]]
        frame_1 = frame_preprocess(frame_1, device)
       
        while True:
            finish=time.time()
            if (finish-begin)>1.9:  
                begin=finish
                js_reply = video_frame(label_html, bbox)
               
                if not js_reply:
                    break 
                frame_2 = js_to_image(js_reply["img"])
              
                frame_2=frame_2[roi_n[1]:roi_n[3],roi_n[0]:roi_n[2]]              
                frame_2 = frame_preprocess(frame_2, device)
                flow_low, flow_up = model(frame_1, frame_2, iters=12, test_mode=True)
                ret = vizualize_flow(frame_1, flow_up, save, counter)              
                pushing_patches=ret
                bbox_array = np.zeros([frame_height,frame_width,4], dtype=np.uint8)
                for (x,y,xx,yy) in pushing_patches:
                      bbox_array = cv2.rectangle(bbox_array,(x,y),(xx,yy),(255,0,0),2)
                bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
                # convert overlay of bbox into bytes
                bbox_bytes = bbox_to_bytes(bbox_array)
                # update bbox
                bbox = bbox_bytes
                ##########################################
                frame_1 = frame_2 
                counter += 1
                           

# Start/Main
Just you need to determine the ROI coordintes, and number of rows and columns patches

In [ ]:
#Pre-trained RAFT model
model_name=home_path+"models/raft-sintel.pth"
ratio_v=1
#Number of rows and columns patches
#patch=[2,4]
#ROI Coordinates
#entrance_2
#roi=[107,55,669,270]

#150 
#roi=[182,100,689,625]
#270 and 280
#roi=[187,165,695,535]
#110
#roi=[187,274,691,432]
parser = ArgumentParser()
parser.add_argument("--model", help="restore checkpoint",type=str, default=model_name)
 
parser.add_argument("--save", action="store_true", help="save demo frames", default=True)
parser.add_argument("--small", action="store_true", help="use small model", default=False)
parser.add_argument("--ratio", type=int, default=ratio_v)
parser.add_argument('--roi','--list', nargs='+',type=int, help='<Required> Set flag',  default=roi)
parser.add_argument('--patch','--list1', nargs='+',type=int, help='<Required> Set flag',  default=patch)
parser.add_argument(
        "--mixed_precision", action="store_true", help="use mixed precision"
    )    

args, unknown = parser.parse_known_args()
start=time.time()
inference(args)
end=time.time()

<IPython.core.display.Javascript object>